In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# p1 calculation

In [10]:
a, b, d, p1 = sp.symbols('alpha beta delta p_1')

p0   = p1/(a*(1-1/b**d))
p0p  = (1-a)*p1
p0dp = a*p1
p2p  = p1*(1-a)*a/((a-1)*(2*a-1))

eq = sp.Eq(p0 + p0p + p0dp + p1 + p2p, 1)
sol = sp.solve(eq, p1)

In [15]:
p0

p_1/(alpha*(1 - 1/beta**delta))

In [14]:
sol[0]

alpha*(2*alpha*beta**delta - 2*alpha - beta**delta + 1)/(3*alpha**2*beta**delta - 3*alpha**2 + 2*alpha - beta**delta)

In [56]:
p1 = sol[0].evalf(subs={d:2})
p1

alpha*(2.0*alpha*beta**2 - 2.0*alpha - beta**2 + 1.0)/(3.0*alpha**2*beta**2 - 3.0*alpha**2 + 2.0*alpha - beta**2)

# f0p0

In [53]:
x, d, b = sp.symbols('x delta beta')
sp.integrate(1/x**2, (x, b, sp.oo))
# sp.integrate(1/x**d, (x))

1/beta

In [64]:
x, d, b, a, g = sp.symbols('x delta beta alpha gamma')

expr = (1/x**d) * ((1/(b+x)**d) + a**2*(1-1/(b+x)**d) + a*(1-a)*(a+(1-a)*g)*(1-1/(b+x)**d))
sp.expand(expr)

alpha**3*gamma/x**delta - alpha**3*gamma/(x**delta*(beta + x)**delta) - alpha**3/x**delta + alpha**3/(x**delta*(beta + x)**delta) - 2*alpha**2*gamma/x**delta + 2*alpha**2*gamma/(x**delta*(beta + x)**delta) + 2*alpha**2/x**delta - 2*alpha**2/(x**delta*(beta + x)**delta) + alpha*gamma/x**delta - alpha*gamma/(x**delta*(beta + x)**delta) + 1/(x**delta*(beta + x)**delta)

In [69]:
expanded = sp.expand(expr).evalf(subs={d:3})
expanded

alpha**3*gamma/x**3 - alpha**3*gamma/(x**3*(beta + x)**3) - alpha**3/x**3 + alpha**3/(x**3*(beta + x)**3) - 2.0*alpha**2*gamma/x**3 + 2.0*alpha**2*gamma/(x**3*(beta + x)**3) + 2.0*alpha**2/x**3 - 2.0*alpha**2/(x**3*(beta + x)**3) + alpha*gamma/x**3 - alpha*gamma/(x**3*(beta + x)**3) + 1/(x**3*(beta + x)**3)

In [70]:
out = sp.integrate(expanded, (x, 0, b))
out

KeyboardInterrupt: 

In [60]:
print_latex(sp.simplify(out))

\begin{cases} \text{NaN} & \text{for}\: \frac{\alpha^{3} \beta^{2} \gamma - \alpha^{3} \beta^{2} - \alpha^{3} \gamma + \alpha^{3} - 2 \alpha^{2} \beta^{2} \gamma + 2 \alpha^{2} \beta^{2} + 2 \alpha^{2} \gamma - 2 \alpha^{2} + \alpha \beta^{2} \gamma - \alpha \gamma + 1}{\beta^{2}} = 0 \\\frac{\infty \beta \left(\alpha^{3} \beta^{2} \gamma - \alpha^{3} \beta^{2} - \alpha^{3} \gamma + \alpha^{3} - 2 \alpha^{2} \beta^{2} \gamma + 2 \alpha^{2} \beta^{2} + 2 \alpha^{2} \gamma - 2 \alpha^{2} + \alpha \beta^{2} \gamma - \alpha \gamma + 1\right) - 1.0 \cdot \left(1.0 \alpha^{3} \beta^{2} \gamma - 1.0 \alpha^{3} \beta^{2} - 0.113705638880109 \alpha^{3} \gamma + 0.113705638880109 \alpha^{3} - 2.0 \alpha^{2} \beta^{2} \gamma + 2.0 \alpha^{2} \beta^{2} + 0.227411277760219 \alpha^{2} \gamma - 0.227411277760219 \alpha^{2} + 1.0 \alpha \beta^{2} \gamma - 0.113705638880109 \alpha \gamma + 0.113705638880109\right) \left|{\frac{\alpha^{3} \beta^{2} \gamma - \alpha^{3} \beta^{2} - \alpha^{3} \gamma + \al

In [61]:
total = sp.simplify((out + 2*a**2/(1-2*a)+3*a-a**2)*p1)
total

nan

In [62]:
total

nan

In [ ]:
# R_f general
a, b, g = sp.symbols('alpha beta gamma')

A_expr = a * (sp.exp(-b) / (a*(1-sp.exp(-b))) + 2 - a + ((2-a)/(1-a)) + ((((1-a)**2)*g)/a) + (a**2/((1-2*a)*(1-a))))
H_expr = (1-a) / (a*(1-sp.exp(-b))) + ((1-a)**2)*(2-g) + a*(1-a)

R_f = A_expr / (A_expr + H_expr)

simplified_Rf = sp.simplify(R_f)
simplified_Rf